In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
import PIL
import tensorflow as tf 
import cv2

from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [34]:

train = pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv',delimiter=',')
train_path = train['Path']
train_label = train['ClassId']
train_path[0]


In [35]:
train_count= len(train)
train_count


# Data Dict

In [37]:
x_train,y_train= [],[]

for path , label in zip(train_path,train_label):
    
    img = cv2.imread('../input/gtsrb-german-traffic-sign/'+path)
    resized = cv2.resize(img,(32,32))
    x_train.append(resized)
    y_train.append(label)





In [47]:
test = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv',delimiter=',')
test_path = test['Path']
test_label = test['ClassId']


x_test,y_test= [],[]

for path , label in zip(test_path,test_label):
    
    img = cv2.imread('../input/gtsrb-german-traffic-sign/'+path)
    resized = cv2.resize(img,(32,32))
    x_test.append(resized)
    y_test.append(label)


# Numpy Array

In [48]:
x_train=np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

# Image Normalization

In [49]:
x_train_normalized = x_train/255
x_test_normalized = x_test/255

In [50]:
x_train.shape

In [51]:
y_train.shape

In [52]:
y_test.shape

In [53]:
x_test.shape

In [55]:
pd.read_csv('../input/gtsrb-german-traffic-sign/Meta.csv',delimiter=',').head(5)

In [56]:

num_classes = len(pd.read_csv('../input/gtsrb-german-traffic-sign/Meta.csv',delimiter=','))
num_classes

# Data Augmentation

In [58]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(32, 32,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

# Model Training

In [65]:
model = models.Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(x_train_normalized, y_train, epochs=30)

In [66]:
model.evaluate(x_test_normalized,y_test)

In [ ]:
model.save(./model.h5) 